In [38]:
import os
import requests
import json
from pprint import pprint
import time
from time import ctime
#import datetime
import csv

from functools import reduce

from IPython.display import clear_output

In [2]:
datapath = "../data/"
outputs = "../outputs/"

In [115]:
api_key = "c0c23ee1793316f86663e645642d8754" # marcelo
#api_key = "e4532171a9f967f4ab9db0763a8bbd87" # hugo

In [4]:
montevideo_geo = [-34.9011, -56.1645]
miraflores_geo = [-12.1111, -77.0316]

In [5]:
rj_meier_geo = [-22.902686, -43.278779]
rj_lagoa_geo = [-22.962273, -43.203564]
rj_barra_geo = [-22.998090, -43.359757]

# links
* https://darksky.net/dev/docs
* https://github.com/bitpixdigital/forecastiopy3

# get data using requests
month = july

SI means International System

In [6]:
latitude=montevideo_geo[0]
longitude=montevideo_geo[1]
asctime= "1561950061"
units = "si"
lang = "en"

In [7]:
url = "https://api.darksky.net/forecast/{}/{},{},{}?units={}&lang={}".format(api_key, latitude, longitude, asctime, units, lang)
print(url)
r = requests.get(url)
data = json.loads(r.text)
pprint(data)

https://api.darksky.net/forecast/c0c23ee1793316f86663e645642d8754/-34.9011,-56.1645,1561950061?units=si&lang=en
{'currently': {'apparentTemperature': 5.74,
               'cloudCover': 0.44,
               'dewPoint': 3.26,
               'humidity': 0.67,
               'icon': 'partly-cloudy-night',
               'ozone': 307,
               'precipIntensity': 0,
               'precipProbability': 0,
               'pressure': 1019.59,
               'summary': 'Partly Cloudy',
               'temperature': 9.14,
               'time': 1561950061,
               'uvIndex': 0,
               'visibility': 10.003,
               'windBearing': 236,
               'windGust': 11.13,
               'windSpeed': 7.29},
 'daily': {'data': [{'apparentTemperatureHigh': 6.63,
                     'apparentTemperatureHighTime': 1562004000,
                     'apparentTemperatureLow': 1.32,
                     'apparentTemperatureLowTime': 1562032800,
                     'apparentTemperat

# print metadata by day and hour

In [126]:
pprint(data['daily'])

{'data': [{'apparentTemperatureHigh': 6.79,
           'apparentTemperatureHighTime': 1562004000,
           'apparentTemperatureLow': 1.3,
           'apparentTemperatureLowTime': 1562032800,
           'apparentTemperatureMax': 6.79,
           'apparentTemperatureMaxTime': 1562004000,
           'apparentTemperatureMin': 1.3,
           'apparentTemperatureMinTime': 1562032800,
           'cloudCover': 0.21,
           'dewPoint': 1.82,
           'humidity': 0.66,
           'icon': 'partly-cloudy-day',
           'moonPhase': 0.97,
           'ozone': 314.9,
           'precipIntensity': 0.0015,
           'precipIntensityMax': 0.0072,
           'precipIntensityMaxTime': 1562004000,
           'precipProbability': 0.07,
           'precipType': 'rain',
           'pressure': 1016.92,
           'summary': 'Windy in the morning and overnight.',
           'sunriseTime': 1561978460,
           'sunsetTime': 1562013906,
           'temperatureHigh': 9.85,
           'temperatureHigh

In [123]:
pprint(data['hourly']['data'][0])

{'apparentTemperature': 5.67,
 'cloudCover': 0.44,
 'dewPoint': 3.06,
 'humidity': 0.66,
 'icon': 'partly-cloudy-night',
 'ozone': 306.9,
 'precipIntensity': 0,
 'precipProbability': 0,
 'pressure': 1019.13,
 'summary': 'Partly Cloudy',
 'temperature': 9.08,
 'time': 1561950000,
 'uvIndex': 0,
 'visibility': 10.003,
 'windBearing': 236,
 'windGust': 11.13,
 'windSpeed': 7.32}


In [131]:
print(data['hourly']['icon'], '\n', data['hourly']['summary'], sep='')

partly-cloudy-day
Partly cloudy throughout the day.


In [120]:
ctime(1562032800)

'Mon Jul  1 23:00:00 2019'

# get data
The first 1,000 API requests you make every day are free of charge.
* https://darksky.net/dev/docs#time-machine-request

In [8]:
def get_weather(api_key, city, time, units, lang):
    latitude = city[0]
    longitude = city[0]
    url = "https://api.darksky.net/forecast/{}/{},{},{}?units={}&lang={}".format(api_key, latitude, longitude, time, units, lang)
    r = requests.get(url)
    data = json.loads(r.text)
    return data

# define data collectors

In [116]:
def save_json(city):
    file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
    city_geo = eval(city+"_geo")
    
    json_list = []
    for i in range(days_range):
        print("RETRIEVING FROM DAY: ",i)
        if i % 28 == 0 or i == days_range-1: clear_output()
        time_iter = int(asctime)+i*daily_seconds
        if int(time_iter) > int(end_time): break
        day_weather = get_weather(api_key, city_geo, time_iter, units, lang)
        json_list.append(day_weather)
        #if i == 1: break
    file_path = os.path.join(datapath,'{}-weather.json'.format(city))
    json.dump(json_list, open(file_path, 'w'))

In [ ]:
test2 = 

In [112]:
def save_csv(city):
    file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
    f = csv.writer(open(file_name, "w"),lineterminator = '\n')
    f.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
    file_name_daily = os.path.join(datapath,'{}-daily_weather.csv'.format(city))
    f_daily = csv.writer(open(file_name_daily, "w"),lineterminator = '\n')
    f_daily.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
    json_data = json.load(open(os.path.join(datapath,'{}-weather.json'.format(city)), 'r'))
    i=0
    for day_weather in json_data:
        time_iter = int(asctime)+i*daily_seconds
        if int(time_iter) > int(end_time): break
            
        date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(day_weather['daily']['data'][0]["time"]))
        f_daily.writerow(
            [date_time,
            day_weather['daily']['data'][0]["cloudCover"],
            day_weather['daily']['data'][0]["humidity"],
            day_weather['daily']['data'][0]["precipIntensity"],
            day_weather['daily']['data'][0]["windSpeed"],
            ])        
            
        hours = day_weather['hourly']['data']        
        for key in hours:
            if key["time"] < int(asctime): continue
            date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(key["time"]))
            f.writerow(
                [date_time,
                key["cloudCover"],
                key["humidity"],
                key["precipIntensity"],
                key["windSpeed"],
                ])
        i+=1

In [72]:
time_tuple = (2019, 8, 27, 0, 0, 0, 0, 0, 0)
end_time = str(int(time.mktime(time_tuple)))
end_time

'1566874800'

In [73]:
time_tuple = (2018, 10, 1, 0, 0, 0, 0, 0, 0)
asctime = str(int(time.mktime(time_tuple))) #"1538362800"
units = "si"
lang = "en"
daily_seconds = 60*60*24 # = number of seconds in a day
days_range = int((int(end_time)-int(asctime))/daily_seconds)
#days_in_month = days_in_month-1

In [74]:
days_range

330

### save json

In [117]:
save_json("rj_meier")

In [119]:
save_json("rj_lagoa")

In [120]:
save_json("rj_barra")

### analyse json

In [79]:
test = json.load(open(os.path.join(datapath,'{}-weather.json'.format("montevideo")), 'r'))
test[0]

In [ ]:
test[1]

### save csv

In [113]:
save_csv("montevideo")

In [114]:
save_csv("miraflores")

In [118]:
save_csv("rj_meier")

In [121]:
save_csv("rj_lagoa")

In [122]:
save_csv("rj_barra")

# get average from a city
cities: Rio de Janeiro

In [123]:
def get_avg(column):
    dvalue = 0
    for k, v in rj_dict.items():
        dvalue = (float(v[i][column]) + dvalue)
    return dvalue/2

In [129]:
rj_dict = {}
for region in ['rj_barra-weather.csv','rj_lagoa-weather.csv', 'rj_meier-weather.csv']:
    file = os.path.join(datapath,region)
    reader = csv.reader(open(file, 'r'))
    key_name = region[:-12]
    rj_dict[key_name] = list(reader)[1:]

In [123]:
def get_avg_json(column):
    dvalue = 0
    for k, v in rj_dict.items():
        dvalue = (float(v[i][column]) + dvalue)
    return dvalue/2

In [148]:
rj_dict_from_json = {}
for region in ['rj_barra-weather.json','rj_lagoa-weather.json', 'rj_meier-weather.json']:
    file = os.path.join(datapath,region)
    reader = json.load(open(file, 'r'))
    key_name = region[:-13]
    rj_dict_from_json[key_name] = list(reader)

In [ ]:
rj_dict_from_json['rj_barra'][0]

In [125]:
rj_dict['rj_barra'][-1]

['2019-08-26 22:00:00', '0.38', '0.78', '0.9387', '5.19']

if you need to ge average using csvs

In [ ]:
city = "rio_de_janeiro"

file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
f = csv.writer(open(file_name, "w"),lineterminator = '\n')
f.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
using 1 key as reference to get len and time
for i in range(len(rj_dict['rj_barra'])):
    date_time = rj_dict['rj_barra'][i][0]
    data_cloud = get_avg(1)
    data_hum = get_avg(2)
    data_prec = get_avg(3)
    f.writerow(
        [date_time,
        data_cloud,
        data_hum,
        data_prec,
        ])

if you need to ge average using jsons

In [128]:
city = "rio_de_janeiro"

file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
f = csv.writer(open(file_name, "w"),lineterminator = '\n')
f.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])

file_name_daily = os.path.join(datapath,'{}-daily_weather.csv'.format(city))
f_daily = csv.writer(open(file_name_daily, "w"),lineterminator = '\n')
f_daily.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
json_data = json.load(open(os.path.join(datapath,'{}-weather.json'.format(city)), 'r'))
i=0
for day_weather in json_data:
    time_iter = int(asctime)+i*daily_seconds
    if int(time_iter) > int(end_time): break
        
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(day_weather['daily']['data'][0]["time"]))
    f_daily.writerow(
        [date_time,
        day_weather['daily']['data'][0]["cloudCover"],
        day_weather['daily']['data'][0]["humidity"],
        day_weather['daily']['data'][0]["precipIntensity"],
        day_weather['daily']['data'][0]["windSpeed"],
        ])        
        
    hours = day_weather['hourly']['data']        
    for key in hours:
        if key["time"] < int(asctime): continue
        date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(key["time"]))
        f.writerow(
            [date_time,
            key["cloudCover"],
            key["humidity"],
            key["precipIntensity"],
            key["windSpeed"],
            ])
    i+=1

In [ ]:
def save_csv(city):
    file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
    f = csv.writer(open(file_name, "w"),lineterminator = '\n')
    f.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
    file_name_daily = os.path.join(datapath,'{}-daily_weather.csv'.format(city))
    f_daily = csv.writer(open(file_name_daily, "w"),lineterminator = '\n')
    f_daily.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
    json_data = json.load(open(os.path.join(datapath,'{}-weather.json'.format(city)), 'r'))
    i=0
    for day_weather in json_data:
        time_iter = int(asctime)+i*daily_seconds
        if int(time_iter) > int(end_time): break
            
        date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(day_weather['daily']['data'][0]["time"]))
        f_daily.writerow(
            [date_time,
            day_weather['daily']['data'][0]["cloudCover"],
            day_weather['daily']['data'][0]["humidity"],
            day_weather['daily']['data'][0]["precipIntensity"],
            day_weather['daily']['data'][0]["windSpeed"],
            ])        
            
        hours = day_weather['hourly']['data']        
        for key in hours:
            if key["time"] < int(asctime): continue
            date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(key["time"]))
            f.writerow(
                [date_time,
                key["cloudCover"],
                key["humidity"],
                key["precipIntensity"],
                key["windSpeed"],
                ])
        i+=1

# compare day to hours data

In [10]:
time_tuple = (2019, 4, 9, 0, 0, 0, 0, 0, 0)
end_time = str(int(time.mktime(time_tuple)))
end_time

'1567479600'

In [12]:
time_tuple = (2018, 4, 8, 0, 0, 0, 0, 0, 0)
asctime = str(int(time.mktime(time_tuple))) #"1538362800"
units = "si"
lang = "en"
daily_seconds = 60*60*24 # = number of seconds in a day
days_range = int((int(end_time)-int(asctime))/daily_seconds)
#days_in_month = days_in_month-1

In [33]:
city = "rj_lagoa"
city_geo = eval(city+"_geo")

time_data = []
var_data = []
time_iter = int(asctime)
day_weather = get_weather(api_key, city_geo, time_iter, units, lang)
hours = day_weather['hourly']['data']
for key in hours:
    if key["time"] < int(asctime): continue
    date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(key["time"]))
    time_data.append(date_time)
    var_data.append(key["precipIntensity"])

In [49]:
temp = reduce(lambda x, y: x + y, var_data) / len(var_data)
print('Average from each hour of day:', temp)

temp = data['daily']['data'][0]['precipIntensity']
print('daily data:', temp)

Average from each hour of day: 2.687826086956522
daily data: 0.0005


daily data: 0.0005


In [51]:
data['daily']

{'data': [{'time': 1560481200,
   'summary': 'Humid and partly cloudy throughout the day.',
   'icon': 'partly-cloudy-day',
   'sunriseTime': 1560504749,
   'sunsetTime': 1560543390,
   'moonPhase': 0.42,
   'precipIntensity': 0.0005,
   'precipIntensityMax': 0.003,
   'precipIntensityMaxTime': 1560502800,
   'precipProbability': 0.05,
   'precipType': 'rain',
   'temperatureHigh': 28.99,
   'temperatureHighTime': 1560524400,
   'temperatureLow': 21.64,
   'temperatureLowTime': 1560592800,
   'apparentTemperatureHigh': 31.31,
   'apparentTemperatureHighTime': 1560524400,
   'apparentTemperatureLow': 22.21,
   'apparentTemperatureLowTime': 1560592800,
   'dewPoint': 22.03,
   'humidity': 0.83,
   'pressure': 1021.65,
   'windSpeed': 2.36,
   'windGust': 4.94,
   'windGustTime': 1560535200,
   'windBearing': 64,
   'cloudCover': 0.35,
   'uvIndex': 6,
   'uvIndexTime': 1560524400,
   'visibility': 10.439,
   'ozone': 241.3,
   'temperatureMin': 22.06,
   'temperatureMinTime': 1560506400,

# tests

### testing time

https://realpython.com/python-time-module/#converting-a-local-time-object-to-seconds

In [16]:
time_tuple = (2018, 10, 1, 0, 0, 0, 0, 0, 0)
int(time.mktime(time_tuple))

1538362800

In [17]:
time_tuple = (2019, 8, 27, 0, 0, 0, 0, 0, 0)
int(time.mktime(time_tuple))

1566874800

In [20]:
time_tuple = (2019, 6, 14, 0, 0, 0, 0, 0, 0)
end_time = str(int(time.mktime(time_tuple)))

In [21]:
end_time

'1560481200'

In [22]:
latitude=rj_lagoa_geo[0]
longitude=rj_lagoa_geo[1]
asctime= end_time
units = "si"
lang = "en"

In [ ]:
url = "https://api.darksky.net/forecast/{}/{},{},{}?units={}&lang={}".format(api_key, latitude, longitude, asctime, units, lang)
print(url)
r = requests.get(url)
data = json.loads(r.text)

# print metadata by day and hour

In [126]:
pprint(data['daily'])

{'data': [{'apparentTemperatureHigh': 6.79,
           'apparentTemperatureHighTime': 1562004000,
           'apparentTemperatureLow': 1.3,
           'apparentTemperatureLowTime': 1562032800,
           'apparentTemperatureMax': 6.79,
           'apparentTemperatureMaxTime': 1562004000,
           'apparentTemperatureMin': 1.3,
           'apparentTemperatureMinTime': 1562032800,
           'cloudCover': 0.21,
           'dewPoint': 1.82,
           'humidity': 0.66,
           'icon': 'partly-cloudy-day',
           'moonPhase': 0.97,
           'ozone': 314.9,
           'precipIntensity': 0.0015,
           'precipIntensityMax': 0.0072,
           'precipIntensityMaxTime': 1562004000,
           'precipProbability': 0.07,
           'precipType': 'rain',
           'pressure': 1016.92,
           'summary': 'Windy in the morning and overnight.',
           'sunriseTime': 1561978460,
           'sunsetTime': 1562013906,
           'temperatureHigh': 9.85,
           'temperatureHigh

In [28]:
str(int(time.mktime((2019, 6, 14, 0, 0, 0, 0, 0, 0))))

'1560481200'

In [29]:
str(int(time.mktime((2019, 6, 14, 18, 0, 0, 0, 0, 0))))

'1560546000'

In [76]:
def save_csv(city):
    file_name = os.path.join(datapath,'{}-weather.csv'.format(city))
    city_geo = eval(city+"_geo")
    
    f = csv.writer(open(file_name, "w"),lineterminator = '\n')
    f.writerow(["time", "cloudCover", "humidity", "precipIntensity", "windSpeed"])
    
    for i in range(days_range):
        print("RETRIEVING FROM DAY: ",i)
        if i % 28 == 0 or i == days_range-1: clear_output()
        time_iter = int(asctime)+i*daily_seconds
        if int(time_iter) > int(end_time): break
        day_weather = get_weather(api_key, city_geo, time_iter, units, lang)
        hours = day_weather['hourly']['data']
        for key in hours:
            if key["time"] < int(asctime): continue
            date_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(key["time"]))
            f.writerow(
                [date_time,
                key["cloudCover"],
                key["humidity"],
                key["precipIntensity"],
                key["windSpeed"],
                ])
        #if i == 1: break